# Sparkify Project Workspace
This workspace contains a tiny subset (128MB) of the full dataset available (12GB). Feel free to use this workspace to build your project, or to explore a smaller subset with Spark before deploying your cluster on the cloud. Instructions for setting up your Spark cluster is included in the last lesson of the Extracurricular Spark Course content.

You can follow the steps below to guide your data analysis and model building portion of this project.

In [61]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf
from pyspark.sql.types import IntegerType
from pyspark.sql import Window
from pyspark.sql.functions import desc
from pyspark.sql.functions import sum as Fsum

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import re

In [2]:
# create a Spark session
spark = SparkSession.builder \
    .master("local") \
    .appName("Sparkify") \
    .getOrCreate()

# Load and Clean Dataset
In this workspace, the mini-dataset file is `mini_sparkify_event_data.json`. Load and clean the dataset, checking for invalid or missing data - for example, records without userids or sessionids. 

In [105]:
sparkify_data = 'mini_sparkify_event_data.json'
df = spark.read.json(sparkify_data)
df.persist()

DataFrame[artist: string, auth: string, firstName: string, gender: string, itemInSession: bigint, lastName: string, length: double, level: string, location: string, method: string, page: string, registration: bigint, sessionId: bigint, song: string, status: bigint, ts: bigint, userAgent: string, userId: string]

In [4]:
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30')

In [5]:
df.count()

286500

In [6]:
df.describe("artist").show()

+-------+------------------+
|summary|            artist|
+-------+------------------+
|  count|            228108|
|   mean| 551.0852017937219|
| stddev|1217.7693079161374|
|    min|               !!!|
|    max| ÃÂlafur Arnalds|
+-------+------------------+



In [7]:
df.describe("userId").show()

+-------+-----------------+
|summary|           userId|
+-------+-----------------+
|  count|           286500|
|   mean|59682.02278593872|
| stddev|109091.9499991047|
|    min|                 |
|    max|               99|
+-------+-----------------+



In [8]:
df.describe("sessionId").show()

+-------+-----------------+
|summary|        sessionId|
+-------+-----------------+
|  count|           286500|
|   mean|1041.526554973822|
| stddev|726.7762634630741|
|    min|                1|
|    max|             2474|
+-------+-----------------+



In [9]:
df = df.dropna(how="any", subset=["userId","sessionId"])

In [10]:
df.count()

286500

In [11]:
df.select("userId").dropDuplicates().sort("userId").show()

+------+
|userId|
+------+
|      |
|    10|
|   100|
|100001|
|100002|
|100003|
|100004|
|100005|
|100006|
|100007|
|100008|
|100009|
|100010|
|100011|
|100012|
|100013|
|100014|
|100015|
|100016|
|100017|
+------+
only showing top 20 rows



In [12]:
df.filter(df["userId"] == "").count()

8346

In [13]:
# all pages
df.select("page").dropDuplicates().collect()

[Row(page='Cancel'),
 Row(page='Submit Downgrade'),
 Row(page='Thumbs Down'),
 Row(page='Home'),
 Row(page='Downgrade'),
 Row(page='Roll Advert'),
 Row(page='Logout'),
 Row(page='Save Settings'),
 Row(page='Cancellation Confirmation'),
 Row(page='About'),
 Row(page='Submit Registration'),
 Row(page='Settings'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='Add to Playlist'),
 Row(page='Add Friend'),
 Row(page='NextSong'),
 Row(page='Thumbs Up'),
 Row(page='Help'),
 Row(page='Upgrade'),
 Row(page='Error'),
 Row(page='Submit Upgrade')]

In [14]:
## blank userId visited pages
df.filter(df.userId == "").select("page").dropDuplicates().collect()

[Row(page='Home'),
 Row(page='About'),
 Row(page='Submit Registration'),
 Row(page='Login'),
 Row(page='Register'),
 Row(page='Help'),
 Row(page='Error')]

Blank userId are guest who just look around the Sparkiy, so we drop them from the dataset

In [106]:
df = df.filter(df.userId != "")
df.count()

278154

In [25]:
df.select("sessionId").dropDuplicates().sort("sessionId").show()

+---------+
|sessionId|
+---------+
|        1|
|        2|
|        3|
|        4|
|        5|
|        6|
|        7|
|        8|
|        9|
|       10|
|       11|
|       12|
|       13|
|       15|
|       16|
|       17|
|       18|
|       19|
|       20|
|       21|
+---------+
only showing top 20 rows



# Exploratory Data Analysis
When you're working with the full dataset, perform EDA by loading a small subset of the data and doing basic manipulations within Spark. In this workspace, you are already provided a small subset of data you can explore.

### Define Churn

Once you've done some preliminary analysis, create a column `Churn` to use as the label for your model. I suggest using the `Cancellation Confirmation` events to define your churn, which happen for both paid and free users. As a bonus task, you can also look into the `Downgrade` events.

### Explore Data
Once you've defined churn, perform some exploratory data analysis to observe the behavior for users who stayed vs users who churned. You can start by exploring aggregates on these two groups of users, observing how much of a specific action they experienced per a certain time unit or number of songs played.

In [74]:
df.filter("page = 'Cancellation Confirmation'").head()

Row(artist=None, auth='Cancelled', firstName='Mason', gender='M', itemInSession=10, lastName='Hart', length=None, level='free', location='Corpus Christi, TX', method='GET', page='Cancellation Confirmation', registration=1533157139000, sessionId=174, song=None, status=200, ts=1539318918000, userAgent='"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='125', cancel=1, Churn=1)

In [107]:
## we flag the cancellation event 1
cancel_event = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())
df = df.withColumn("cancel", cancel_event("page"))
df.head()

Row(artist='Martha Tilston', auth='Logged In', firstName='Colin', gender='M', itemInSession=50, lastName='Freeman', length=277.89016, level='paid', location='Bakersfield, CA', method='PUT', page='NextSong', registration=1538173362000, sessionId=29, song='Rockpools', status=200, ts=1538352117000, userAgent='Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) Gecko/20100101 Firefox/31.0', userId='30', cancel=0)

Since Cancellation can happen to paid and free users, we flag these users 1 on all event records before their cancellation and 0 otherwise by creating Churn column.

In [108]:
windowval = Window.partitionBy("userId").orderBy(desc("ts")).rangeBetween(Window.unboundedPreceding, 0)
df = df.withColumn("Churn", Fsum("cancel").over(windowval))

In [109]:
df.select(["userId", "firstname", "ts", "page","method","sessionId","song", "level", "Churn"]).where(df.userId == "100014").sort("ts").collect()

[Row(userId='100014', firstname='Rodrigo', ts=1539177136000, page='NextSong', method='PUT', sessionId=37, song='Welcome To The World of The Plastic Beach (Feat. Snoop Dogg and Hypnotic Brass Ensemble)', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177351000, page='NextSong', method='PUT', sessionId=37, song='Odessa', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177476000, page='NextSong', method='PUT', sessionId=37, song='Amapola', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177698000, page='NextSong', method='PUT', sessionId=37, song='Exogenesis: Symphony Part 1 [Overture]', level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177705000, page='Help', method='GET', sessionId=37, song=None, level='paid', Churn=1),
 Row(userId='100014', firstname='Rodrigo', ts=1539177956000, page='NextSong', method='PUT', sessionId=37, song='Sehr kosmisch', level='paid', Churn=1),
 Row(userId='100

Exploratory Data Analysis on groups who stayed and churned

In [110]:
# About 16% churn rate
df.groupby("Churn").count().show()

+-----+------+
|Churn| count|
+-----+------+
|    0|233290|
|    1| 44864|
+-----+------+



In [111]:
cancel_event = udf(lambda x: 1 if x == "NextSong" else 0, IntegerType())
df = df.withColumn("listen", cancel_event("page"))

On average, users who stayed listened to 676 songs and users who churned listened to 431 songs

In [127]:
df.filter(df.Churn == 0).groupby("userId","listen").agg({"listen":"sum"}).agg({"sum(listen)":"avg"}).show()

+-----------------+
| avg(sum(listen))|
+-----------------+
|555.6927536231884|
+-----------------+



In [128]:
df.filter(df.Churn == 1).groupby("userId","listen").agg({"listen":"sum"}).agg({"sum(listen)":"avg"}).show()

+-----------------+
| avg(sum(listen))|
+-----------------+
|349.9423076923077|
+-----------------+



In [114]:
thumbsdown_event = udf(lambda x: 1 if x == "Thumbs Down" else 0, IntegerType())
df = df.withColumn("NumThumbsDown", thumbsdown_event("page"))
help_event = udf(lambda x: 1 if x == "Help" else 0, IntegerType())
df = df.withColumn("NumHelp", help_event("page"))
error_event = udf(lambda x: 1 if x == "Error" else 0, IntegerType())
df = df.withColumn("NumError", error_event("page"))
ad_event = udf(lambda x: 1 if x == "Roll Advert" else 0, IntegerType())
df = df.withColumn("NumAd", ad_event("page"))


In [115]:
df.filter(df.Churn == 0).groupby("userId","NumThumbsDown").agg({"NumThumbsDown":"count"}).agg({"count(NumThumbsDown)":"avg"}).show()

+-------------------------+
|avg(count(NumThumbsDown))|
+-------------------------+
|                   711.25|
+-------------------------+



In [116]:
df.filter(df.Churn == 1).groupby("userId","NumThumbsDown").agg({"NumThumbsDown":"count"}).agg({"count(NumThumbsDown)":"avg"}).show()

+-------------------------+
|avg(count(NumThumbsDown))|
+-------------------------+
|                   448.64|
+-------------------------+



In [117]:
df.filter(df.Churn == 0).groupby("userId","NumHelp").agg({"NumHelp":"count"}).agg({"count(NumHelp)":"avg"}).show()

+-------------------+
|avg(count(NumHelp))|
+-------------------+
|  724.5031055900621|
+-------------------+



In [118]:
df.filter(df.Churn == 1).groupby("userId","NumHelp").agg({"NumHelp":"count"}).agg({"count(NumHelp)":"avg"}).show()

+-------------------+
|avg(count(NumHelp))|
+-------------------+
|  472.2526315789474|
+-------------------+



In [119]:
df.filter(df.Churn == 0).groupby("userId","NumError").agg({"NumError":"count"}).agg({"count(NumError)":"avg"}).show()

+--------------------+
|avg(count(NumError))|
+--------------------+
|   870.4850746268656|
+--------------------+



In [120]:
df.filter(df.Churn == 1).groupby("userId","NumError").agg({"NumError":"count"}).agg({"count(NumError)":"avg"}).show()

+--------------------+
|avg(count(NumError))|
+--------------------+
|   606.2702702702703|
+--------------------+



In [121]:
df.filter(df.Churn == 0).groupby("userId","NumAd").agg({"NumAd":"count"}).agg({"count(NumAd)":"avg"}).show()

+-----------------+
|avg(count(NumAd))|
+-----------------+
| 706.939393939394|
+-----------------+



In [122]:
df.filter(df.Churn == 1).groupby("userId","NumAd").agg({"NumAd":"count"}).agg({"count(NumAd)":"avg"}).show()

+------------------+
| avg(count(NumAd))|
+------------------+
|439.84313725490193|
+------------------+



# Feature Engineering
Once you've familiarized yourself with the data, build out the features you find promising to train your model on. To work with the full dataset, you can follow the following steps.
- Write a script to extract the necessary features from the smaller subset of data
- Ensure that your script is scalable, using the best practices discussed in Lesson 3
- Try your script on the full data set, debugging your script if necessary

If you are working in the classroom workspace, you can just extract features based on the small subset of data contained here. Be sure to transfer over this work to the larger dataset when you work on your Spark cluster.

# Modeling
Split the full dataset into train, test, and validation sets. Test out several of the machine learning methods you learned. Evaluate the accuracy of the various models, tuning parameters as necessary. Determine your winning model based on test accuracy and report results on the validation set. Since the churned users are a fairly small subset, I suggest using F1 score as the metric to optimize.

# Final Steps
Clean up your code, adding comments and renaming variables to make the code easier to read and maintain. Refer to the Spark Project Overview page and Data Scientist Capstone Project Rubric to make sure you are including all components of the capstone project and meet all expectations. Remember, this includes thorough documentation in a README file in a Github repository, as well as a web app or blog post.